## 实验一 

使用`pytorch`还原 LeNet 模型

In [1]:
import torch

In [2]:
import torch.nn as nn
import torch.nn.functional as F

### 定义模型

In [3]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5,padding=2)     # 28 28 6
        self.pool1 = nn.AvgPool2d(2,2)     # 14 14 6
        self.conv2 = nn.Conv2d(6,16,5)      # 10 10 16
        self.pool2 = nn.AvgPool2d(2,2)     # 5 5 16
        self.fc1 = nn.Linear(16*5*5 ,120)  # 120
        self.fc2 = nn.Linear(120,84)      # 84
        self.fc3 = nn.Linear(84,10)
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
net = LeNet5()
print(net)

LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### load MNIST dataset

In [5]:
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [6]:
batch_size = 128
training_data = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.MNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)


train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size,shuffle=False)

In [7]:
print(training_data[0][0].shape)

torch.Size([1, 28, 28])


In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
learning_rate = 1e-2
epochs = 20
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

In [10]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net, loss_fn, optimizer)
    test_loop(test_dataloader, net, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302099  [    0/60000]
loss: 0.166252  [12800/60000]
loss: 0.173810  [25600/60000]
loss: 0.196820  [38400/60000]
loss: 0.085899  [51200/60000]
Test Error: 
 Accuracy: 95.9%, Avg loss: 0.001012 

Epoch 2
-------------------------------
loss: 0.136572  [    0/60000]
loss: 0.054141  [12800/60000]
loss: 0.125088  [25600/60000]
loss: 0.030951  [38400/60000]
loss: 0.104834  [51200/60000]
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.000395 

Epoch 3
-------------------------------
loss: 0.084271  [    0/60000]
loss: 0.047590  [12800/60000]
loss: 0.008119  [25600/60000]
loss: 0.015903  [38400/60000]
loss: 0.020729  [51200/60000]
Test Error: 
 Accuracy: 98.0%, Avg loss: 0.000506 

Epoch 4
-------------------------------
loss: 0.071333  [    0/60000]
loss: 0.086644  [12800/60000]
loss: 0.011294  [25600/60000]
loss: 0.001538  [38400/60000]
loss: 0.071178  [51200/60000]
Test Error: 
 Accuracy: 98.7%, Avg loss: 0.000353 

Epoch 5
------------------------

In [14]:
# 保存模型
torch.save(net,'./models/lab1_1.pt')